# Check processed corpus

## Check short posts in white supremacist corpus to see if express the ideology

In [ ]:
# Load white supremacist corpus
import pandas as pd

path = '../tmp/white_supremacist_train_corpus.pkl'
data = pd.read_pickle(path)
data.info()

In [ ]:
# View short posts across domains
from IPython.display import display

pd.set_option('display.max_colwidth', None)

for domain in data.domain.unique():
    print(domain)
    domain_data = data.query('domain==@domain')
    short = domain_data[domain_data.text.str.split().str.len() <= 10]
    display(short.sample(10))

In [ ]:
# View distribution of post length overall
import plotly.express as px
pd.options.plotting.backend = 'plotly'

lengths = data.text.str.split().str.len()

In [ ]:
data['length'] = lengths

In [ ]:
# remove outliers
sampled = lengths.sample(int(1e6))
sampled[sampled < 200].hist()

In [ ]:
s.domain.value_counts()

In [ ]:
# Plot lengths across domains
s = data[data.length < 100].sample(int(1e6))
# s.hist(column='length')
px.histogram(s, x='length', color='domain', barmode='overlay')

In [ ]:
# Plot lengths across non-forum domains
s = data.query('length < 100 and domain != "forum"')
# s.hist(column='length')
px.histogram(s, x='length', color='domain', barmode='overlay')

In [ ]:
cdf = lengths.value_counts().sort_index().cumsum()/len(lengths)
cdf

In [ ]:
px.line(cdf[:100])

In [ ]:
# cdfs = {domain: data[data.domain==domain]['length'].value_counts().sort_index().cumsum()/len(data) for domain in data.domain.unique()}
# cdfs = pd.concat([pd.DataFrame(
#     {'cumsum': data[data.domain==domain]['length'].value_counts().sort_index().cumsum()/len(data), 'domain': domain}) for domain in data.domain.unique()
#                  ])
cdfs = pd.concat([pd.DataFrame(
    data[data.domain==domain]['length'].value_counts().sort_index().cumsum()/len(data[data.domain==domain])).assign(domain=domain).rename_axis('value').reset_index() for domain in data.domain.unique()
                 ])
cdfs.info()
cdfs[cdfs.value <= 100]

In [ ]:
px.line(cdfs[cdfs.value <= 100], x='value', y='length', color='domain')

# Corpus stats

In [9]:
# Load data
import pandas as pd

path = '../tmp/white_supremacist_train_corpus.pkl'
# path = '../tmp/antiracist_train_corpus.pkl'
# path = '../tmp/domain_test_corpus.pkl'
data = pd.read_pickle(path)
# path = '../data/corpora/white_supremacist_corpus.json'
# data = pd.read_json(path, orient='table')
# print(len(data))
# data.columns
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3299122 entries, qian2018_0 to pruden2022_161
Data columns (total 6 columns):
 #   Column     Dtype              
---  ------     -----              
 0   text       object             
 1   dataset    object             
 2   source     object             
 3   domain     object             
 4   timestamp  datetime64[ns, UTC]
 5   num_words  int64              
dtypes: datetime64[ns, UTC](1), int64(1), object(4)
memory usage: 176.2+ MB


In [10]:
# Post and word counts total and per domain
data['num_words'] = data.text.str.split().str.len()
stats = data.groupby('domain').agg({'num_words': ['count', 'sum', 'mean']})
stats.columns = ['post_count', 'word_count', 'avg_post_length']
stats

total = pd.DataFrame({'post_count': len(data), 'word_count': data.num_words.sum()}, index=['total'])
total

pd.concat([stats,total])

,post_count,word_count,avg_post_length
chat,18739,425654,22.714873
forum,3171564,206524897,65.117682
long-form,38036,11420987,300.267825
tweet,70783,1253683,17.711640
total,3299122,219625221,NaN


In [11]:
data.dataset.value_counts()

papasavva2020       1901408
stormfront           706232
jokubausaite2020     415921
ironmarch            148003
qian2018              66294
calderon2021          25685
patriotfront          18739
pruden2022            12351
elsherief2021          4489
Name: dataset, dtype: int64

In [ ]:
data.groupby(['domain', 'dataset'])['text'].count()

In [ ]:
# counts = data.groupby(['domain', 'dataset'])['text'].count().reset_index()
counts = data.groupby(['domain', 'dataset'])['text'].count()
counts

In [ ]:
data.groupby(['domain', 'dataset', 'source'])['text'].count()

In [ ]:
# Can't figure out how to sort wihin domains
# counts.sort_values(['domain', 'dataset'], ascending=False).groupby('domain').groups()
# counts.sort_values(['domain', 'dataset'], ascending=False).groupby('domain')['text'].sum()

In [ ]:
# Num of posts from each dataset (maybe should do log scale)
pd.options.plotting.backend = 'plotly'
vc = data.dataset.value_counts()
print(vc)
vc.plot.bar(text=vc.values)

In [ ]:
# Num of posts from each dataset (maybe should do log scale)
pd.options.plotting.backend = 'plotly'
vc = data.dataset.value_counts()
print(vc)
vc.plot.bar(text=vc.values)

In [ ]:
# Number of words
data['num_words'] = data.text.str.split().str.len()
# data.head()

word_count =  data.groupby('dataset')['num_words'].sum()
display(word_count)
word_count.sort_values(ascending=False).plot.bar().show()

# Total posts and words
print(f'{len(data)} posts')
print(f'{word_count.sum()} words')

In [ ]:
import plotly.express as px

# By domain
dataset_info = pd.DataFrame([
    {'dataset': 'qian2018', 'domain': 'tweet/short propaganda'},
    {'dataset': 'elsherief2021', 'domain': 'tweet/short propaganda'},
    {'dataset': 'degibert2019', 'domain': 'forum'},
    {'dataset': 'patriotfront', 'domain': 'chat'},
    {'dataset': 'alatawi2021', 'domain': 'tweet/short propaganda'},
    {'dataset': 'adl_heatmap', 'domain': 'tweet/short propaganda'},
    {'dataset': 'ironmarch', 'domain': 'forum'},
    {'dataset': '4chan', 'domain': 'forum'},
    {'dataset': 'stormfront', 'domain': 'forum'},
    {'dataset': 'calderon2021', 'domain': 'long-form'},
    {'dataset': 'pruden2022', 'domain': 'long-form'},
]).set_index('dataset')
dataset_info

# data.join(dataset_info, on='dataset').domain.value_counts().plot.bar(
#     title="Number of posts by domain", 
#     labels=dict(index='domain', value='number of posts'),
# )

vc = data.join(dataset_info, on='dataset').domain.value_counts()
fig = px.bar(vc, title="Number of posts by domain", 
       labels=dict(index='domain', value='number of posts'),
        text = [f'{val}<br>{val/vc.sum(): .1%}' for val in vc.values],
    )
fig.update_layout(showlegend=False)

In [ ]:
num_words = data.join(dataset_info, on='dataset').groupby('domain')['num_words'].sum().sort_values(ascending=False)
fig = px.bar(num_words,
    title="Number of words by domain", 
    labels=dict(index='domain', value='number of words'),
    text = [f'{val}<br>{val/num_words.sum(): .1%}' for val in num_words.values],
       # log_y=True
    )
fig.update_layout(showlegend=False)

In [ ]:
# Investigate date ranges
print(len(data))
print(data.timestamp.count())

In [ ]:
pd.options.plotting.backend = 'plotly'
data.timestamp.sample(int(1e6)).hist()
# data.timestamp.plot.hist()

In [ ]:
start_date = pd.Timestamp(year=2000, month=1, day=1, tz='utc')
data.query('timestamp > @start_date').timestamp.sample(int(1e5)).hist()

In [ ]:
# Bin or group by month
import plotly.express as px
# pd.options.plotting.backend = 'plotly'

start_date = pd.Timestamp(year=2000, month=1, day=1, tz='utc')
merged =  data.join(dataset_info, on='dataset').query('timestamp > @start_date').sample(int(1e6))
# merged.hist(column='timestamp', by='domain')
# merged.hist(column=['timestamp'])
px.histogram(merged, x='timestamp', color='domain', barmode=')

In [ ]:
# Bin or group by month
px.histogram(merged, x='timestamp', color='domain', barmode='overlay')

In [ ]:
import plotly.express as px

px.histogram(data.query('domain=="forum"').sample(int(1e5)), x='timestamp', color='dataset', barmode='overlay')